In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import math

# Initialize MediaPipe for hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Initialize Haarcascade for face and smile detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

# Screen Size
screen_w, screen_h = pyautogui.size()

# Mode Control
click_mode = False
mouse_enabled = True

# ------------------ Helper Functions ------------------
def get_distance(p1, p2):
    return math.hypot(p2[0] - p1[0], p2[1] - p1[1])

def is_left_click(landmarks):
    return get_distance(landmarks[4], landmarks[8]) < 0.05

def is_right_click(landmarks):
    return get_distance(landmarks[8], landmarks[12]) < 0.05

def is_double_click(landmarks):
    return get_distance(landmarks[8], landmarks[16]) < 0.05

def is_screenshot(landmarks):
    return get_distance(landmarks[8], landmarks[20]) < 0.05

# ---------------- Webcam Capture ----------------
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, c = frame.shape

    # ------------ Face Detection -----------------
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    emotion = "Neutral"

    if len(faces) == 0:
        emotion = "No Face"
        click_mode = False
        mouse_enabled = False
        cv2.putText(frame, "No Face - Mouse Disabled", (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                    0.8, (0, 0, 255), 2)

    for (x, y, fw, fh) in faces:
        face_roi = gray[y:y+fh, x:x+fw]

        # ✅ Stricter smile detection (for visible teeth only)
        smiles = smile_cascade.detectMultiScale(face_roi, scaleFactor=1.5, minNeighbors=25)

        smile_detected = False
        for (sx, sy, sw, sh) in smiles:
            if sw > fw * 0.4 and sh > fh * 0.15:  # Detect only wide, visible teeth smiles
                smile_detected = True
                break

        if smile_detected:
            emotion = "Smiling (Teeth)"
            click_mode = True
            mouse_enabled = True
            cv2.putText(frame, " Smile - Click Mode ON", (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                        0.8, (0, 255, 0), 2)
        else:
            emotion = "Neutral"
            click_mode = False
            mouse_enabled = True
            cv2.putText(frame, "Face Detected - Move Mode ON", (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                        0.8, (255, 255, 0), 2)

        cv2.rectangle(frame, (x, y), (x + fw, y + fh), (255, 0, 0), 2)

    # ------------- Hand Tracking -----------------
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks and mouse_enabled:
        for hand_landmark in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmark, mp_hands.HAND_CONNECTIONS)

            landmark_list = [(lm.x, lm.y) for lm in hand_landmark.landmark]

            # Mouse Movement
            index_x = int(landmark_list[8][0] * w)
            index_y = int(landmark_list[8][1] * h)

            screen_x = screen_w * landmark_list[8][0]
            screen_y = screen_h * landmark_list[8][1]
            pyautogui.moveTo(screen_x, screen_y)

            cv2.circle(frame, (index_x, index_y), 10, (255, 0, 255), cv2.FILLED)

            # --------- Click Gestures when click_mode is True ----------
            if click_mode:
                if is_left_click(landmark_list):
                    cv2.putText(frame, "Left Click", (50, 100), cv2.FONT_HERSHEY_SIMPLEX,
                                1, (0, 255, 0), 2)
                    pyautogui.click()
                    pyautogui.sleep(1)

                elif is_right_click(landmark_list):
                    cv2.putText(frame, "Right Click", (50, 150), cv2.FONT_HERSHEY_SIMPLEX,
                                1, (255, 0, 0), 2)
                    pyautogui.click(button='right')
                    pyautogui.sleep(1)

                elif is_double_click(landmark_list):
                    cv2.putText(frame, "Double Click", (50, 200), cv2.FONT_HERSHEY_SIMPLEX,
                                1, (0, 255, 255), 2)
                    pyautogui.doubleClick()
                    pyautogui.sleep(1)

                elif is_screenshot(landmark_list): 
                    cv2.putText(frame, "Screenshot", (50, 250), cv2.FONT_HERSHEY_SIMPLEX,
                                1, (0, 0, 255), 2)
                    pyautogui.screenshot("screenshot.png")
                    pyautogui.sleep(1)

    elif not mouse_enabled:
        cv2.putText(frame, "Angry/No Face - Mouse Disabled", (50, 100), cv2.FONT_HERSHEY_SIMPLEX,
                    0.8, (0, 0, 255), 2)

    cv2.imshow("Virtual Mouse with Hand + Emotion Control", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
